# Infleeting Lead Times — Compound Deep Dive

Analyzes actual days per infleeting phase (ordered → produced → arrived → tech prep → ready → in_subscription) by brand and compound.

**Data source**: `datawarehouse-304513.cars_ops_postgres_public.car_events`  
**API source**: `https://api-ops-cars.finn.auto` (ops-cars API, documented in `finn-auto/redocly`)

In [1]:
from google.cloud import bigquery
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

client = bigquery.Client(project='datawarehouse-304513')

def run_query(sql):
    return client.query(sql).to_dataframe()

## 1. Brand / Compound Configuration

Current compound assignments from the ops-cars lead times config:

In [2]:
COMPOUND_MAP = [
    ('Mazda', 'cat_zuelpich'), ('Ford', 'cat_zuelpich'), ('Polestar', 'cat_zuelpich'),
    ('Honda', 'akb_kitzingen'), ('Kia', 'blg_bremerhaven'), ('Kia', 'blg_saalanderdonau'),
    ('Nissan', 'blg_neuss'), ('Mazda', 'blg_duisburg'),
    ('Cadillac', 'ceva_grevenbroich'), ('Cadillac', 'ceva_bad_salzungen'),
    ('Hyundai', 'blg_bremerhaven'), ('Hyundai', 'mosolf_rackwitz'), ('Hyundai', 'mosolf_cuxhaven'),
    ('Jeep', 'mosolf_goessnitz'), ('Jeep', 'mosolf_kippenheim'), ('Jeep', 'mosolf_etzin'),
    ('Fiat', 'mosolf_kippenheim'), ('Fiat', 'mosolf_etzin'),
    ('Alfa Romeo', 'mosolf_kippenheim'),
    ('Mercedes-Benz', 'neuhaus_gmbh_via_bald'),
    ('MG', 'carserviceerkens_wachtendonk'), ('MG', 'mosolf_wilhelmshaven'),
    ('BYD', 'mosolf_wilhelmshaven'),
    ('BMW', 'akb_kitzingen'), ('MINI', 'akb_kitzingen'),
    ('Dacia', 'blg_duisburg'), ('Renault', 'blg_duisburg'),
    ('Peugeot', 'mosolf_kippenheim'), ('Peugeot', 'mosolf_etzin'), ('Peugeot', 'siebrecht_uslar'),
    ('Citroen', 'mosolf_kippenheim'), ('DS', 'mosolf_kippenheim'),
    ('Opel', 'mosolf_etzin'), ('Opel', 'mosolf_kippenheim'), ('Opel', 'siebrecht_uslar'),
    ('Toyota', 'blg_saalanderdonau'),
    ('Seat', 'akb_zoerbig'), ('Cupra', 'akb_zoerbig'),
    ('Skoda', 'mosolf_rackwitz'),
    ('Audi', 'akb_kitzingen'),
]

# Build SQL filter
filter_tuples = ', '.join(f"('{b}', '{c}')" for b, c in COMPOUND_MAP)
print(f'{len(COMPOUND_MAP)} brand/compound combinations configured')

40 brand/compound combinations configured


## 2. Compound Lead Times — Avg & Median per Phase

In [3]:
lead_times_query = f"""
WITH events_all AS (
  SELECT
    e.car_id, e.name, e.time,
    JSON_VALUE(e.snapshot, '$.oem') AS oem,
    JSON_VALUE(e.snapshot, '$.infleeting_compound_id') AS compound_id
  FROM `datawarehouse-304513.cars_ops_postgres_public.car_events` e
  WHERE e.name IN (
    'car_ordered','car_state_changed_produced','car_state_changed_arrived_from_supplier',
    'car_state_changed_tech_prep_done','car_state_changed_ready_to_deliver','car_state_changed_in_subscription'
  )
),
cars_active_2025 AS (
  SELECT DISTINCT car_id FROM events_all
  WHERE time >= '2025-01-01' AND time < '2026-01-01'
),
filtered AS (
  SELECT e.* FROM events_all e
  INNER JOIN cars_active_2025 c ON e.car_id = c.car_id
),
state_changes AS (
  SELECT car_id, name, time, oem, compound_id,
    LAG(time) OVER (PARTITION BY car_id ORDER BY time) AS prev_time
  FROM filtered
),
with_days AS (
  SELECT *, DATE_DIFF(DATE(time), DATE(prev_time), DAY) AS days_in_status
  FROM state_changes WHERE prev_time IS NOT NULL
),
compound_lookup AS (
  SELECT DISTINCT car_id, compound_id FROM filtered
  WHERE name = 'car_state_changed_arrived_from_supplier' AND compound_id IS NOT NULL
)
SELECT
  d.oem AS brand,
  COALESCE(cl.compound_id, d.compound_id) AS compound,
  COUNT(DISTINCT d.car_id) AS cars,

  ROUND(AVG(IF(d.name='car_state_changed_produced', d.days_in_status, NULL)),1) AS avg_ordered_to_produced,
  APPROX_QUANTILES(IF(d.name='car_state_changed_produced', d.days_in_status, NULL), 100 IGNORE NULLS)[OFFSET(50)] AS med_ordered_to_produced,

  ROUND(AVG(IF(d.name='car_state_changed_arrived_from_supplier', d.days_in_status, NULL)),1) AS avg_produced_to_arrived,
  APPROX_QUANTILES(IF(d.name='car_state_changed_arrived_from_supplier', d.days_in_status, NULL), 100 IGNORE NULLS)[OFFSET(50)] AS med_produced_to_arrived,

  ROUND(AVG(IF(d.name='car_state_changed_tech_prep_done', d.days_in_status, NULL)),1) AS avg_arrived_to_techprep,
  APPROX_QUANTILES(IF(d.name='car_state_changed_tech_prep_done', d.days_in_status, NULL), 100 IGNORE NULLS)[OFFSET(50)] AS med_arrived_to_techprep,

  ROUND(AVG(IF(d.name='car_state_changed_ready_to_deliver', d.days_in_status, NULL)),1) AS avg_techprep_to_ready,
  APPROX_QUANTILES(IF(d.name='car_state_changed_ready_to_deliver', d.days_in_status, NULL), 100 IGNORE NULLS)[OFFSET(50)] AS med_techprep_to_ready,

  ROUND(AVG(IF(d.name='car_state_changed_in_subscription', d.days_in_status, NULL)),1) AS avg_ready_to_sub,
  APPROX_QUANTILES(IF(d.name='car_state_changed_in_subscription', d.days_in_status, NULL), 100 IGNORE NULLS)[OFFSET(50)] AS med_ready_to_sub

FROM with_days d
LEFT JOIN compound_lookup cl ON d.car_id = cl.car_id
WHERE (d.oem, COALESCE(cl.compound_id, d.compound_id)) IN ({filter_tuples})
GROUP BY 1, 2
ORDER BY brand, compound
"""

df = run_query(lead_times_query)
print(f'{len(df)} brand/compound combos, {df["cars"].sum():,} total car transitions')
df

40 brand/compound combos, 48,549 total car transitions


/Users/jan.heiselbetz/venvs/jupyter-py313/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,brand,compound,cars,avg_ordered_to_produced,med_ordered_to_produced,avg_produced_to_arrived,med_produced_to_arrived,avg_arrived_to_techprep,med_arrived_to_techprep,avg_techprep_to_ready,med_techprep_to_ready,avg_ready_to_sub,med_ready_to_sub
0,Alfa Romeo,mosolf_kippenheim,268,180.0,188,10.5,0,18.0,2,66.3,32,27.3,18
1,Audi,akb_kitzingen,2484,139.3,124,67.7,68,25.2,19,23.3,7,12.3,8
2,BMW,akb_kitzingen,4446,31.5,28,88.9,85,13.9,2,16.5,9,10.8,8
3,BYD,mosolf_wilhelmshaven,3078,14.9,0,34.3,29,4.4,0,5.9,0,35.9,29
4,Cadillac,ceva_bad_salzungen,18,0.0,0,11.1,11,7.9,0,31.0,18,10.7,7
5,Cadillac,ceva_grevenbroich,16,0.0,0,22.1,24,10.9,0,42.9,49,6.0,7
6,Citroen,mosolf_kippenheim,795,125.2,116,14.4,1,6.8,5,13.4,13,20.0,15
7,Cupra,akb_zoerbig,1628,67.8,64,53.0,42,9.1,0,25.8,19,13.6,9
8,DS,mosolf_kippenheim,150,89.0,69,14.2,14,10.7,5,19.4,25,47.6,36
9,Dacia,blg_duisburg,2051,14.6,0,52.5,62,19.1,21,15.2,9,18.9,14


## 3. Heatmap — Median Days per Phase

In [4]:
# Prepare heatmap data
df['label'] = df['brand'] + ' @ ' + df['compound'].str.replace('_', ' ')

phases = [
    ('med_ordered_to_produced', 'Ordered→Produced'),
    ('med_produced_to_arrived', 'Produced→Arrived'),
    ('med_arrived_to_techprep', 'Arrived→Tech Prep'),
    ('med_techprep_to_ready', 'Tech Prep→Ready'),
    ('med_ready_to_sub', 'Ready→Subscription'),
]

heat_df = df[['label', 'cars'] + [p[0] for p in phases]].copy()
heat_df.columns = ['label', 'cars'] + [p[1] for p in phases]

# Sort by total median days
phase_cols = [p[1] for p in phases]
heat_df['total'] = heat_df[phase_cols].sum(axis=1)
heat_df = heat_df.sort_values('total', ascending=True)

fig = px.imshow(
    heat_df[phase_cols].values,
    labels=dict(x='Phase', y='Brand @ Compound', color='Median Days'),
    x=phase_cols,
    y=heat_df['label'].values,
    color_continuous_scale='RdYlGn_r',
    aspect='auto',
    text_auto=True,
)
fig.update_layout(
    title='Infleeting Lead Times — Median Days per Phase (2025)',
    height=max(600, len(heat_df) * 22),
    width=900,
    font=dict(size=11),
)
fig.show()

## 4. Stacked Bar — Total Median Lead Time by Brand/Compound

In [5]:
# Stacked horizontal bar chart
bar_df = heat_df.sort_values('total', ascending=True).copy()

colors = {
    'Ordered→Produced': '#4e79a7',
    'Produced→Arrived': '#f28e2b',
    'Arrived→Tech Prep': '#e15759',
    'Tech Prep→Ready': '#76b7b2',
    'Ready→Subscription': '#59a14f',
}

fig = go.Figure()
for phase in phase_cols:
    fig.add_trace(go.Bar(
        y=bar_df['label'],
        x=bar_df[phase].fillna(0),
        name=phase,
        orientation='h',
        marker_color=colors[phase],
        text=bar_df[phase].fillna(0).astype(int),
        textposition='inside',
    ))

fig.update_layout(
    barmode='stack',
    title='Total Infleeting Lead Time — Median Days (2025)',
    xaxis_title='Days',
    height=max(600, len(bar_df) * 22),
    width=1000,
    legend=dict(orientation='h', yanchor='bottom', y=1.02),
    font=dict(size=11),
)
fig.show()

## 5. Avg vs Median Skew — Identifying Outlier-Heavy Combos

In [6]:
# Show where avg >> median (heavy right tail / outliers)
skew_data = []
for _, row in df.iterrows():
    for avg_col, med_col, phase_name in [
        ('avg_ordered_to_produced', 'med_ordered_to_produced', 'Ordered→Produced'),
        ('avg_produced_to_arrived', 'med_produced_to_arrived', 'Produced→Arrived'),
        ('avg_arrived_to_techprep', 'med_arrived_to_techprep', 'Arrived→Tech Prep'),
        ('avg_techprep_to_ready', 'med_techprep_to_ready', 'Tech Prep→Ready'),
        ('avg_ready_to_sub', 'med_ready_to_sub', 'Ready→Subscription'),
    ]:
        avg_val = row[avg_col]
        med_val = row[med_col]
        if pd.notna(avg_val) and pd.notna(med_val) and med_val > 0:
            skew_data.append({
                'brand': row['brand'],
                'compound': row['compound'],
                'phase': phase_name,
                'avg': avg_val,
                'median': med_val,
                'skew_ratio': round(avg_val / med_val, 1),
                'cars': row['cars'],
            })

skew_df = pd.DataFrame(skew_data)
print('Top 15 most skewed (avg/median ratio) — indicates outlier problems:')
skew_df.sort_values('skew_ratio', ascending=False).head(15)

Top 15 most skewed (avg/median ratio) — indicates outlier problems:


,brand,compound,phase,avg,median,skew_ratio,cars
120,Nissan,blg_neuss,Ordered→Produced,29.6,1,29.6,1240
61,Hyundai,mosolf_cuxhaven,Ordered→Produced,39.4,2,19.7,997
23,Citroen,mosolf_kippenheim,Produced→Arrived,14.4,1,14.4,795
1,Alfa Romeo,mosolf_kippenheim,Arrived→Tech Prep,18.0,2,9.0,268
11,BMW,akb_kitzingen,Arrived→Tech Prep,13.9,2,7.0,4446
99,MG,mosolf_wilhelmshaven,Tech Prep→Ready,13.7,2,6.8,3610
157,Polestar,cat_zuelpich,Tech Prep→Ready,29.8,5,6.0,583
81,Jeep,mosolf_kippenheim,Produced→Arrived,22.9,4,5.7,620
46,Fiat,mosolf_kippenheim,Tech Prep→Ready,159.8,39,4.1,55
138,Opel,siebrecht_uslar,Tech Prep→Ready,80.0,20,4.0,470


## 6. Single Car Deep Dive

In [7]:
def car_deep_dive(car_id: str):
    """Fetch and display the full infleeting timeline for a single car."""
    query = f"""
    WITH state_changes AS (
      SELECT
        car_id, name, time,
        JSON_VALUE(snapshot, '$.oem') AS oem,
        JSON_VALUE(snapshot, '$.model') AS model,
        JSON_VALUE(snapshot, '$.infleeting_compound_id') AS compound_id,
        JSON_VALUE(snapshot, '$.status') AS status,
        LAG(time) OVER (ORDER BY time) AS prev_time,
        LAG(name) OVER (ORDER BY time) AS prev_name
      FROM `datawarehouse-304513.cars_ops_postgres_public.car_events`
      WHERE car_id = '{car_id}'
        AND name IN (
          'car_ordered','car_state_changed_produced','car_state_changed_arrived_from_supplier',
          'car_state_changed_tech_prep_done','car_state_changed_ready_to_deliver','car_state_changed_in_subscription'
        )
    )
    SELECT
      name AS event,
      time,
      oem, model, compound_id, status,
      DATE_DIFF(DATE(time), DATE(prev_time), DAY) AS days_in_prev_status
    FROM state_changes
    ORDER BY time
    """
    result = run_query(query)
    if len(result) == 0:
        print(f'No state change events found for car {car_id}')
        return None

    oem = result.iloc[0]['oem']
    model = result.iloc[0]['model']
    compound = result.iloc[-1]['compound_id']
    print(f'Car {car_id}: {oem} {model} @ {compound}')
    print(f'Total: {(result["time"].max() - result["time"].min()).days}d from ordered to {result.iloc[-1]["status"]}')
    print()
    return result[['event', 'time', 'days_in_prev_status', 'status']]

# Example: the Nissan Qashqai we analyzed earlier
car_deep_dive('zxmotnpu')

Car zxmotnpu: Nissan Qashqai @ blg_neuss
Total: 45d from ordered to in_subscription



/Users/jan.heiselbetz/venvs/jupyter-py313/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning:

BigQuery Storage module not found, fetch data with the REST endpoint instead.



,event,time,days_in_prev_status,status
0,car_ordered,2025-12-30 07:17:55.642000+00:00,<NA>,ordered
1,car_state_changed_produced,2025-12-30 07:17:55.869000+00:00,0,produced
2,car_state_changed_arrived_from_supplier,2026-01-07 06:18:54.461000+00:00,8,arrived_from_supplier
3,car_state_changed_tech_prep_done,2026-01-26 08:20:47.248000+00:00,19,tech_preparation_done
4,car_state_changed_ready_to_deliver,2026-02-02 08:00:55.471000+00:00,7,ready_to_deliver
5,car_state_changed_in_subscription,2026-02-13 09:53:59.273000+00:00,11,in_subscription


## 7. Monthly Trend — Fleet-Wide Throughput

In [8]:
trend_query = """
SELECT
  FORMAT_DATE('%Y-%m', DATE(time)) AS month,
  CASE name
    WHEN 'car_state_changed_produced' THEN 'Produced'
    WHEN 'car_state_changed_arrived_from_supplier' THEN 'Arrived'
    WHEN 'car_state_changed_tech_prep_done' THEN 'Tech Prep Done'
    WHEN 'car_state_changed_ready_to_deliver' THEN 'Ready to Deliver'
    WHEN 'car_state_changed_in_subscription' THEN 'In Subscription'
  END AS status,
  COUNT(DISTINCT car_id) AS cars
FROM `datawarehouse-304513.cars_ops_postgres_public.car_events`
WHERE name IN (
  'car_state_changed_produced','car_state_changed_arrived_from_supplier',
  'car_state_changed_tech_prep_done','car_state_changed_ready_to_deliver',
  'car_state_changed_in_subscription'
)
AND time >= '2025-01-01' AND time < '2026-01-01'
GROUP BY 1, 2
ORDER BY 1, 2
"""

trend_df = run_query(trend_query)

fig = px.line(
    trend_df, x='month', y='cars', color='status',
    title='Monthly Cars Reaching Each Infleeting Phase (2025)',
    labels={'cars': 'Cars', 'month': 'Month'},
    markers=True,
)
fig.update_layout(height=450, width=900, legend=dict(orientation='h', yanchor='bottom', y=1.02))
fig.show()

/Users/jan.heiselbetz/venvs/jupyter-py313/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning:

BigQuery Storage module not found, fetch data with the REST endpoint instead.



## 8. Bottleneck Detector — Worst Phases per Compound

In [9]:
# Find the worst phase for each brand/compound (highest median)
bottlenecks = []
for _, row in df.iterrows():
    worst_phase = None
    worst_val = 0
    for med_col, phase_name in [
        ('med_ordered_to_produced', 'Ordered→Produced'),
        ('med_produced_to_arrived', 'Produced→Arrived'),
        ('med_arrived_to_techprep', 'Arrived→Tech Prep'),
        ('med_techprep_to_ready', 'Tech Prep→Ready'),
        ('med_ready_to_sub', 'Ready→Subscription'),
    ]:
        val = row[med_col]
        if pd.notna(val) and val > worst_val:
            worst_val = val
            worst_phase = phase_name
    bottlenecks.append({
        'brand': row['brand'],
        'compound': row['compound'],
        'cars': row['cars'],
        'bottleneck_phase': worst_phase,
        'median_days': worst_val,
    })

bn_df = pd.DataFrame(bottlenecks).sort_values('median_days', ascending=False)

fig = px.bar(
    bn_df.head(20),
    x='median_days',
    y=bn_df.head(20).apply(lambda r: f"{r['brand']} @ {r['compound']}", axis=1),
    color='bottleneck_phase',
    orientation='h',
    title='Top 20 Bottlenecks — Worst Phase per Brand/Compound (Median Days)',
    labels={'median_days': 'Median Days', 'y': ''},
)
fig.update_layout(height=550, width=900, yaxis=dict(autorange='reversed'))
fig.show()

## 9. Expected vs. Actual Lead Times — Backtesting Accuracy

### How the API actually calculates `Available From`

The ops-cars API (`infleeting-lead-times.ts`) uses a **two-layer system**:

```
Available From = start_date + compound_lead_time_days
```

| Car Status | start_date | compound_lead_time | **Owner** |
|---|---|---|---|
| `ordered` / `produced` | **max(Fleet ETA, today)** | `OrderedLeadTimes` / `ProducedLeadTimes` | **Fleet** (ETA) + **Infleeting** (compound) |
| `arrived_from_supplier` | **today** (or anchor date) | `ArrivedFromSupplierLeadTimes` | **Infleeting** |
| `tech_prep_done` | **today** (or anchor date) | `TechPreparationDoneLeadTimes` | **Infleeting** |
| `ready_to_deliver` | **today** (or anchor date) | `ReadyToDeliverLeadTimes` | **Infleeting** |

### Two independent accuracy questions

1. **Fleet Layer (ETA accuracy)**: How long does `ordered → arrived_from_supplier` actually take? Fleet predicts this via OEM Delivery Plans. We don't have per-car ETA snapshots in BigQuery, but we CAN measure the actual duration distribution.

2. **Infleeting Layer (compound accuracy)**: How accurate is `t_arrived` (predicted days from compound arrival → subscription)? We CAN directly compare this against actual `arrived → in_subscription` medians.

### Why ordered = produced for most brands

For nearly every brand, `t_ordered = t_produced` (identical compound lead times). This means the API assumes **0 days of compound lead time difference** between ordered and produced — the real time between these states is entirely determined by Fleet ETA. Only Kia (56→42d) and Nissan (35→21d) have different values.

In [ ]:
# API compound lead times (t-minus: remaining days until subscription from each status)
# Source: ops-cars-repo/src/domain/availabilities/lead-times/*.ts
# Using worst-case paths: unpaid, no SA number, outbound check not done
# Conversion: 1 week = 7 days, N working days ≈ ceil(N × 7/5) calendar days
#
# IMPORTANT: For ordered/produced, these values are NOT phase durations.
# They are compound processing times added to Fleet ETA: Available From = max(ETA, now) + t_value
# For arrived/techprep/ready, they ARE remaining days: Available From = now + t_value

API_LEAD_TIMES = [
    # (brand, compound, t_ordered, t_produced, t_arrived, t_techprep, t_ready, notes, source)
    # --- MAZDA (InfleetingProcess.MAZDA) ---
    ('Mazda', 'cat_zuelpich', 35, 35, 28, 28, 28, 'Unpaid; arrived=4w unpaid, techprep=4w unpaid, ready=4w mazda unpaid',
     'ordered:5w, produced:5w, arrived:getMazdaLeadTime(unpaid)=4w, techprep:AUDI/MAZDA(unpaid)=4w, ready:cat_zuelpich+MAZDA(unpaid)=4w'),
    ('Mazda', 'blg_duisburg', None, None, None, None, None, 'Only 1 car, no reference data', ''),
    # --- FORD (InfleetingProcess.PCDO) ---
    ('Ford', 'cat_zuelpich', 28, 28, 21, 14, 5, '',
     'ordered:PCDO=4w, produced:PCDO=4w, arrived:PCDO=3w, techprep:PCDO=wd(10)≈14d, ready:cat_zuelpich+PCDO=wd(3)≈5d'),
    # --- POLESTAR ---
    ('Polestar', 'cat_zuelpich', 35, 35, 21, 7, 5, '',
     'ordered:5w, produced:5w, arrived:POLESTAR=3w, techprep:POLESTAR=1w, ready:cat_zuelpich=wd(3)≈5d'),
    # --- HONDA (InfleetingProcess.MKM_HUBER) ---
    ('Honda', 'akb_kitzingen', 35, 35, 21, 10, 7, 'Outbound check not done',
     'ordered:MKM_HUBER=5w, produced:MKM_HUBER=5w, arrived:MKM_HUBER=3w, techprep:default=10d, ready:akb_kitzingen(no_outbound)=wd(5)≈7d'),
    # --- KIA ---
    ('Kia', 'blg_bremerhaven', 56, 42, 21, 10, 7, '',
     'ordered:KIA=8w, produced:KIA=6w, arrived:KIA=3w, techprep:default=10d, ready:blg_bremerhaven+KIA=wd(5)≈7d'),
    ('Kia', 'blg_saalanderdonau', 56, 42, 21, 10, 7, '',
     'ordered:KIA=8w, produced:KIA=6w, arrived:KIA=3w, techprep:default=10d, ready:blg_saalanderdonau+KIA=wd(5)≈7d'),
    # --- NISSAN (InfleetingProcess.MKM_HUBER) ---
    ('Nissan', 'blg_neuss', 35, 21, 21, 10, 7, '',
     'ordered:MKM_HUBER=5w, produced:MKM_HUBER+blg_neuss=3w, arrived:MKM_HUBER=3w, techprep:default=10d, ready:blg_neuss+MKM_HUBER=wd(5)≈7d'),
    # --- CADILLAC ---
    ('Cadillac', 'ceva_grevenbroich', 21, 21, 14, 10, 7, 'Outbound check not done',
     'ordered:CADILLAC=3w, produced:CADILLAC=3w, arrived:CADILLAC=14d, techprep:default=10d, ready:ceva+CADILLAC(no_outbound)=wd(5)≈7d'),
    ('Cadillac', 'ceva_bad_salzungen', 21, 21, 14, 10, 7, 'Outbound check not done',
     'ordered:CADILLAC=3w, produced:CADILLAC=3w, arrived:CADILLAC=14d, techprep:default=10d, ready:ceva+CADILLAC(no_outbound)=wd(5)≈7d'),
    # --- HYUNDAI ---
    ('Hyundai', 'blg_bremerhaven', 42, 42, 35, 10, 7, '',
     'ordered:HYUNDAI_VIA_BLG=6w, produced:HYUNDAI_VIA_BLG=6w, arrived:HYUNDAI_VIA_BLG=5w, techprep:default=10d, ready:blg_bremerhaven+HYUNDAI_BLG=wd(5)≈7d'),
    ('Hyundai', 'mosolf_rackwitz', 42, 42, 21, 10, 5, '',
     'ordered:HYUNDAI_VIA_MOSOLF=6w, produced:HYUNDAI_VIA_MOSOLF=6w, arrived:HYUNDAI_VIA_MOSOLF=3w, techprep:default=10d, ready:mosolf_rackwitz+HYUNDAI_MOSOLF=wd(3)≈5d'),
    ('Hyundai', 'mosolf_cuxhaven', 42, 42, 21, 10, 7, '',
     'ordered:HYUNDAI_VIA_MOSOLF=6w, produced:HYUNDAI_VIA_MOSOLF=6w, arrived:HYUNDAI_VIA_MOSOLF=3w, techprep:default=10d, ready:mosolf_cuxhaven+HYUNDAI_MOSOLF=wd(5)≈7d'),
    # --- FCA (Jeep, Fiat, Alfa Romeo) ---
    ('Jeep', 'mosolf_goessnitz', 21, 21, 21, 10, 9, 'No SA number',
     'ordered:FCA=3w, produced:FCA=3w, arrived:FCA=3w, techprep:default=10d, ready:mosolf no_SA=wd(6)≈9d'),
    ('Jeep', 'mosolf_kippenheim', 21, 21, 21, 10, 9, 'No SA number',
     'ordered:FCA=3w, produced:FCA=3w, arrived:FCA=3w, techprep:default=10d, ready:mosolf_kippenheim no_SA=wd(6)≈9d'),
    ('Jeep', 'mosolf_etzin', 21, 21, 21, 10, 9, 'No SA number',
     'ordered:FCA=3w, produced:FCA=3w, arrived:FCA=3w, techprep:default=10d, ready:mosolf_etzin no_SA=wd(6)≈9d'),
    ('Fiat', 'mosolf_kippenheim', 21, 21, 21, 10, 9, 'No SA number',
     'ordered:FCA=3w, produced:FCA=3w, arrived:FCA=3w, techprep:default=10d, ready:mosolf_kippenheim no_SA=wd(6)≈9d'),
    ('Fiat', 'mosolf_etzin', 21, 21, 21, 10, 9, 'No SA number',
     'ordered:FCA=3w, produced:FCA=3w, arrived:FCA=3w, techprep:default=10d, ready:mosolf_etzin no_SA=wd(6)≈9d'),
    ('Alfa Romeo', 'mosolf_kippenheim', 21, 21, 21, 10, 9, 'No SA number',
     'ordered:FCA=3w, produced:FCA=3w, arrived:FCA=3w, techprep:default=10d, ready:mosolf_kippenheim no_SA=wd(6)≈9d'),
    # --- MERCEDES ---
    ('Mercedes-Benz', 'neuhaus_gmbh_via_bald', 28, 28, 14, 21, 7, 'Unpaid path',
     'ordered:MERCEDES_BALD=4w, produced:MERCEDES_BALD=4w, arrived:MERCEDES_BALD=2w, techprep:MERCEDES_BALD(unpaid)=3w, ready:neuhaus+MERCEDES(unpaid)=3w→but wd(5)≈7d'),
    # --- MG ---
    ('MG', 'carserviceerkens_wachtendonk', 21, 21, 14, 10, 7, 'Often skips tech_prep',
     'ordered:MG=3w, produced:MG=3w, arrived:MG@erkens=2w, techprep:default=10d, ready:erkens+MG=wd(5)≈7d'),
    ('MG', 'mosolf_wilhelmshaven', 21, 21, 21, 10, 9, 'No SA; often skips tech_prep',
     'ordered:MG=3w, produced:MG=3w, arrived:MG@mosolf_whaven=3w, techprep:default=10d, ready:mosolf_whaven+MG no_SA=wd(6)≈9d'),
    # --- BYD ---
    ('BYD', 'mosolf_wilhelmshaven', 35, 35, 21, 10, 14, 'No SA + no release; often skips tech_prep',
     'ordered:BYD=5w, produced:BYD=5w, arrived:BYD=3w, techprep:default=10d, ready:mosolf_whaven+BYD no_SA no_release=wd(10)≈14d'),
    # --- BMW/MINI ---
    ('BMW', 'akb_kitzingen', 45, 45, 14, 10, 7, 'supplierStatus 170 default location; often skips tech_prep',
     'ordered:BMW@170=45d, produced:BMW@170=45d, arrived:BMW=2w, techprep:default=10d, ready:akb_kitzingen(no_outbound)=wd(5)≈7d'),
    ('MINI', 'akb_kitzingen', 45, 45, 14, 10, 7, 'supplierStatus 170 default; often skips tech_prep',
     'ordered:BMW@170=45d, produced:BMW@170=45d, arrived:BMW=2w, techprep:default=10d, ready:akb_kitzingen(no_outbound)=wd(5)≈7d'),
    # --- DACIA/RENAULT ---
    ('Dacia', 'blg_duisburg', 35, 35, 35, 14, 14, 'Unpaid path',
     'ordered:RENAULT=5w, produced:RENAULT=5w, arrived:getRenault(unpaid)=5w, techprep:RENAULT(unpaid)=2w, ready:blg_duisburg+RENAULT(unpaid)=2w'),
    ('Renault', 'blg_duisburg', 35, 35, 35, 14, 14, 'Unpaid path',
     'ordered:RENAULT=5w, produced:RENAULT=5w, arrived:getRenault(unpaid)=5w, techprep:RENAULT(unpaid)=2w, ready:blg_duisburg+RENAULT(unpaid)=2w'),
    # --- STELLANTIS (Peugeot, Citroen, DS, Opel) ---
    ('Peugeot', 'mosolf_kippenheim', 28, 28, 21, 9, 9, 'No SA number',
     'ordered:STELLANTIS=4w, produced:STELLANTIS=4w, arrived:STELLANTIS=3w, techprep:STELLANTIS=9d, ready:mosolf_kippenheim no_SA=wd(6)≈9d'),
    ('Peugeot', 'mosolf_etzin', 28, 28, 21, 9, 9, 'No SA number',
     'ordered:STELLANTIS=4w, produced:STELLANTIS=4w, arrived:STELLANTIS=3w, techprep:STELLANTIS=9d, ready:mosolf_etzin no_SA=wd(6)≈9d'),
    ('Peugeot', 'siebrecht_uslar', 28, 28, 21, 9, 7, '',
     'ordered:STELLANTIS=4w, produced:STELLANTIS=4w, arrived:STELLANTIS=3w, techprep:SIEBRECHT=9d, ready:default=1w'),
    ('Citroen', 'mosolf_kippenheim', 28, 28, 21, 9, 9, 'No SA number',
     'ordered:STELLANTIS=4w, produced:STELLANTIS=4w, arrived:STELLANTIS=3w, techprep:STELLANTIS=9d, ready:mosolf_kippenheim no_SA=wd(6)≈9d'),
    ('DS', 'mosolf_kippenheim', 28, 28, 21, 9, 9, 'No SA number',
     'ordered:STELLANTIS=4w, produced:STELLANTIS=4w, arrived:STELLANTIS=3w, techprep:STELLANTIS=9d, ready:mosolf_kippenheim no_SA=wd(6)≈9d'),
    ('Opel', 'mosolf_etzin', 28, 28, 21, 9, 9, 'No SA number',
     'ordered:STELLANTIS=4w, produced:STELLANTIS=4w, arrived:STELLANTIS=3w, techprep:STELLANTIS=9d, ready:mosolf_etzin no_SA=wd(6)≈9d'),
    ('Opel', 'mosolf_kippenheim', 28, 28, 21, 9, 9, 'No SA number',
     'ordered:STELLANTIS=4w, produced:STELLANTIS=4w, arrived:STELLANTIS=3w, techprep:STELLANTIS=9d, ready:mosolf_kippenheim no_SA=wd(6)≈9d'),
    ('Opel', 'siebrecht_uslar', 28, 28, 21, 9, 7, '',
     'ordered:STELLANTIS=4w, produced:STELLANTIS=4w, arrived:STELLANTIS=3w, techprep:SIEBRECHT=9d, ready:default=1w'),
    # --- TOYOTA ---
    ('Toyota', 'blg_saalanderdonau', 28, 28, 21, 10, 7, '',
     'ordered:TOYOTA=4w, produced:TOYOTA=4w, arrived:TOYOTA=3w, techprep:default=10d, ready:blg_saalanderdonau+TOYOTA=wd(5)≈7d'),
    # --- VW GROUP (FESER / FESER_SEAT_ILLINGEN / SKODA_VIA_AH_KOCH / AUDI) ---
    ('Seat', 'akb_zoerbig', 49, 49, 35, 28, 7, 'Unpaid; outbound not done',
     'ordered:FESER_SEAT(unpaid)=7w, produced:FESER_SEAT(unpaid)=7w, arrived:FESER_SEAT(unpaid)=5w, techprep:FESER_SEAT(unpaid)=4w, ready:akb_zoerbig(no_outbound)=wd(5)≈7d'),
    ('Cupra', 'akb_zoerbig', 49, 49, 35, 28, 7, 'Unpaid; outbound not done',
     'ordered:FESER_SEAT(unpaid)=7w, produced:FESER_SEAT(unpaid)=7w, arrived:FESER_SEAT(unpaid)=5w, techprep:FESER_SEAT(unpaid)=4w, ready:akb_zoerbig(no_outbound)=wd(5)≈7d'),
    ('Skoda', 'mosolf_rackwitz', 49, 49, 35, 28, 5, 'Unpaid',
     'ordered:SKODA_AH_KOCH(unpaid)=7w, produced:SKODA_AH_KOCH(unpaid)=7w, arrived:SKODA_AH_KOCH(unpaid)=5w, techprep:SKODA_AH_KOCH(unpaid)=4w, ready:mosolf_rackwitz=wd(3)≈5d'),
    ('Audi', 'akb_kitzingen', 49, 49, 35, 21, 7, 'Unpaid; outbound not done',
     'ordered:AUDI(unpaid)=7w, produced:AUDI(unpaid)=7w, arrived:AUDI(unpaid)=5w→but getAudiLeadTime=5w=35d→wait techprep:AUDI(unpaid)=4w→but 3w=21d, ready:akb_kitzingen(no_outbound)=wd(5)≈7d'),
]

# Build t-minus DataFrame
tminus_df = pd.DataFrame(API_LEAD_TIMES, columns=[
    'brand', 'compound',
    't_ordered', 't_produced', 't_arrived', 't_techprep', 't_ready',
    'notes', 'api_source'
])

# Derive expected phase durations from t-minus differences
has_data = tminus_df['t_ordered'].notna()
tminus_valid = tminus_df[has_data].copy()

# Phase duration = t_minus[current] - t_minus[next]; last phase = t_ready itself
tminus_valid['exp_ordered_to_produced'] = tminus_valid['t_ordered'] - tminus_valid['t_produced']
tminus_valid['exp_produced_to_arrived'] = tminus_valid['t_produced'] - tminus_valid['t_arrived']
tminus_valid['exp_arrived_to_techprep'] = tminus_valid['t_arrived'] - tminus_valid['t_techprep']
tminus_valid['exp_techprep_to_ready'] = tminus_valid['t_techprep'] - tminus_valid['t_ready']
tminus_valid['exp_ready_to_sub'] = tminus_valid['t_ready']

# NOTE: exp_ordered_to_produced = 0 for most brands because ordered/produced
# have the same compound lead time. This is BY DESIGN — the API doesn't predict
# OEM production time; it relies on Fleet ETA for that.

exp_cols = ['brand', 'compound',
    'exp_ordered_to_produced', 'exp_produced_to_arrived',
    'exp_arrived_to_techprep', 'exp_techprep_to_ready', 'exp_ready_to_sub',
    'notes',
    't_ordered', 't_produced', 't_arrived', 't_techprep', 't_ready']

no_data = tminus_df[~has_data][['brand', 'compound', 'notes',
    't_ordered', 't_produced', 't_arrived', 't_techprep', 't_ready']].copy()
for col in ['exp_ordered_to_produced', 'exp_produced_to_arrived',
            'exp_arrived_to_techprep', 'exp_techprep_to_ready', 'exp_ready_to_sub']:
    no_data[col] = None

expected_df = pd.concat([tminus_valid[exp_cols], no_data[exp_cols]], ignore_index=True)

print(f'{len(expected_df)} brand/compound combinations with API lead times')
print(f'\nKey: t_ordered/t_produced = compound lead time added to ETA')
print(f'     t_arrived/t_techprep/t_ready = remaining days to subscription from today')
print(f'\nBrands where ordered ≠ produced (API gives different values):')
diff = tminus_valid[tminus_valid['t_ordered'] != tminus_valid['t_produced']]
if len(diff) > 0:
    print(diff[['brand', 'compound', 't_ordered', 't_produced']].to_string(index=False))
else:
    print('  None — all brands have identical ordered/produced values')

print(f'\nDerived phase durations (t-minus differences):')
expected_df[expected_df['exp_ordered_to_produced'].notna()][
    ['brand', 'compound', 'exp_ordered_to_produced', 'exp_produced_to_arrived',
     'exp_arrived_to_techprep', 'exp_techprep_to_ready', 'exp_ready_to_sub']
].head(15)

### 9.1 Fleet Layer — ETA Accuracy & Ordered→Arrived Duration (Fleet-owned)

The Fleet team owns the ETA prediction: how long from `ordered` (or `produced`) until the car physically arrives at the compound (`arrived_from_supplier`).

**Data available**: The `car_events` snapshot JSON contains `estimated_arrival_from_supplier_date` for every event, and the event `car_estimated_arrival_from_compound_date_updated` fires when ETA changes. This lets us:

1. **Compare predicted ETA vs actual arrival date** — the direct measure of Fleet accuracy
2. **Track ETA drift** — how often and by how much ETAs change over a car's lifecycle
3. **Show actual ordered→arrived durations** — how long Fleet's phase actually takes
4. **Identify brands where Fleet error dominates** total prediction error

In [ ]:
# === FLEET LAYER: ETA Accuracy + Ordered→Arrived duration ===

# --- PART A: ETA Accuracy (predicted vs actual arrival) ---
# Extract the FIRST ETA set for each car and compare to actual arrival date

eta_query = f"""
WITH eta_events AS (
  -- Get the first ETA prediction per car (earliest car_estimated_arrival_from_compound_date_updated event)
  SELECT
    car_id,
    JSON_VALUE(snapshot, '$.oem') AS oem,
    JSON_VALUE(snapshot, '$.infleeting_compound_id') AS compound_id,
    DATE(JSON_VALUE(snapshot, '$.estimated_arrival_from_supplier_date')) AS predicted_eta,
    DATE(time) AS eta_set_date,
    ROW_NUMBER() OVER (PARTITION BY car_id ORDER BY time ASC) AS rn
  FROM 
  WHERE name = 'car_estimated_arrival_from_compound_date_updated'
    AND time >= '2025-01-01' AND time < '2026-01-01'
    AND JSON_VALUE(snapshot, '$.estimated_arrival_from_supplier_date') IS NOT NULL
),
first_eta AS (
  SELECT * FROM eta_events WHERE rn = 1
),
actual_arrival AS (
  -- Get actual arrival date per car
  SELECT
    car_id,
    DATE(time) AS actual_arrival_date
  FROM 
  WHERE name = 'car_state_changed_arrived_from_supplier'
    AND time >= '2025-01-01' AND time < '2026-01-01'
),
eta_accuracy AS (
  SELECT
    f.car_id, f.oem, f.compound_id,
    f.predicted_eta, f.eta_set_date,
    a.actual_arrival_date,
    DATE_DIFF(a.actual_arrival_date, f.predicted_eta, DAY) AS eta_error_days
  FROM first_eta f
  INNER JOIN actual_arrival a ON f.car_id = a.car_id
  WHERE (f.oem, f.compound_id) IN ({filter_tuples})
)
SELECT
  oem AS brand,
  compound_id AS compound,
  COUNT(*) AS cars_with_eta,
  ROUND(AVG(eta_error_days), 1) AS avg_eta_error,
  APPROX_QUANTILES(eta_error_days, 100)[OFFSET(50)] AS med_eta_error,
  APPROX_QUANTILES(eta_error_days, 100)[OFFSET(25)] AS p25_eta_error,
  APPROX_QUANTILES(eta_error_days, 100)[OFFSET(75)] AS p75_eta_error,
  APPROX_QUANTILES(eta_error_days, 100)[OFFSET(5)] AS p5_eta_error,
  APPROX_QUANTILES(eta_error_days, 100)[OFFSET(95)] AS p95_eta_error,
  ROUND(AVG(ABS(eta_error_days)), 1) AS mae_eta,
  COUNTIF(ABS(eta_error_days) <= 7) AS within_1_week,
  COUNTIF(ABS(eta_error_days) <= 14) AS within_2_weeks,
  COUNTIF(eta_error_days > 0) AS arrived_late,
  COUNTIF(eta_error_days < 0) AS arrived_early,
  COUNTIF(eta_error_days = 0) AS arrived_on_time
FROM eta_accuracy
GROUP BY 1, 2
ORDER BY mae_eta DESC
"""

eta_df = run_query(eta_query)

# Also count cars WITHOUT any ETA (ordered but no ETA event)
no_eta_query = f"""
WITH ordered_cars AS (
  SELECT DISTINCT car_id,
    JSON_VALUE(snapshot, '$.oem') AS oem,
    JSON_VALUE(snapshot, '$.infleeting_compound_id') AS compound_id
  FROM 
  WHERE name = 'car_ordered'
    AND time >= '2025-01-01' AND time < '2026-01-01'
),
cars_with_eta AS (
  SELECT DISTINCT car_id
  FROM 
  WHERE name = 'car_estimated_arrival_from_compound_date_updated'
    AND time >= '2025-01-01' AND time < '2026-01-01'
)
SELECT
  o.oem AS brand,
  o.compound_id AS compound,
  COUNT(*) AS total_ordered,
  COUNTIF(e.car_id IS NOT NULL) AS has_eta,
  COUNTIF(e.car_id IS NULL) AS no_eta,
  ROUND(SAFE_DIVIDE(COUNTIF(e.car_id IS NULL), COUNT(*)) * 100, 1) AS pct_no_eta
FROM ordered_cars o
LEFT JOIN cars_with_eta e ON o.car_id = e.car_id
WHERE (o.oem, o.compound_id) IN ({filter_tuples})
GROUP BY 1, 2
ORDER BY pct_no_eta DESC
"""

no_eta_df = run_query(no_eta_query)

print('=== FLEET ETA ACCURACY (first ETA prediction vs actual arrival) ===')
print(f'Cars with ETA + actual arrival: {eta_df[cars_with_eta].sum():,}
')

if len(eta_df) > 0:
    total_cars = eta_df['cars_with_eta'].sum()
    within_1w = eta_df['within_1_week'].sum()
    within_2w = eta_df['within_2_weeks'].sum()
    print(f'Overall: {within_1w/total_cars*100:.0f}% within ±1 week, {within_2w/total_cars*100:.0f}% within ±2 weeks')
    print(f'Mean absolute error: {(eta_df[mae_eta] * eta_df[cars_with_eta]).sum() / total_cars:.1f} days')
    print(f'Late arrivals (after ETA): {eta_df[arrived_late].sum()/total_cars*100:.0f}%')
    print(f'Early arrivals (before ETA): {eta_df[arrived_early].sum()/total_cars*100:.0f}%
')

    print(f'{"Brand":<20} {"Compound":<30} {"Cars":>5} {"MAE":>5} {"Med Err":>8} {"P25-P75":>12} {"±1wk%":>6} {"±2wk%":>6}')
    print('-' * 93)
    for _, r in eta_df.sort_values('mae_eta', ascending=False).iterrows():
        w1pct = r['within_1_week'] / r['cars_with_eta'] * 100 if r['cars_with_eta'] > 0 else 0
        w2pct = r['within_2_weeks'] / r['cars_with_eta'] * 100 if r['cars_with_eta'] > 0 else 0
        print(f'{r["brand"]:<20} {r["compound"]:<30} {r["cars_with_eta"]:>5} {r["mae_eta"]:>5.1f} {r["med_eta_error"]:>+7}d  [{r["p25_eta_error"]:>+4},{r["p75_eta_error"]:>+4}]d {w1pct:>5.0f}% {w2pct:>5.0f}%')

print(f'
=== CARS WITHOUT ANY ETA (Fleet blind spot) ===')
print(f'These cars have max(ETA, now) = now, making Available From pure guesswork
')
high_no_eta = no_eta_df[no_eta_df['pct_no_eta'] > 20].sort_values('pct_no_eta', ascending=False)
if len(high_no_eta) > 0:
    for _, r in high_no_eta.iterrows():
        print(f'  {r["brand"]:<20} @ {r["compound"]:<25} {r["no_eta"]:>4}/{r["total_ordered"]:>4} cars ({r["pct_no_eta"]:.0f}%) have NO ETA')
else:
    print('  All brands have ETA coverage >80%')

# --- ETA accuracy chart ---
if len(eta_df) > 0:
    eta_chart = eta_df.copy()
    eta_chart['label'] = eta_chart['brand'] + ' @ ' + eta_chart['compound'].str.replace('_', ' ')
    eta_chart = eta_chart.sort_values('med_eta_error')

    fig = go.Figure()
    # Error bars: P25-P75
    fig.add_trace(go.Bar(
        y=eta_chart['label'], x=eta_chart['med_eta_error'],
        orientation='h', name='Median ETA Error (days)',
        marker_color=eta_chart['med_eta_error'].apply(lambda x: '#e45756' if x > 0 else '#4c78a8'),
        error_x=dict(
            type='data',
            symmetric=False,
            array=(eta_chart['p75_eta_error'] - eta_chart['med_eta_error']).clip(lower=0),
            arrayminus=(eta_chart['med_eta_error'] - eta_chart['p25_eta_error']).clip(lower=0)
        ),
        text=eta_chart['med_eta_error'].apply(lambda x: f'{x:+d}d'),
        textposition='outside'
    ))
    fig.add_vline(x=0, line_dash='dash', line_color='gray')
    fig.update_layout(
        title='Fleet ETA Accuracy: Median Error (days late vs early) with P25-P75 range',
        xaxis_title='Days (positive = arrived LATE, negative = arrived EARLY)',
        height=max(500, len(eta_chart) * 22),
        margin=dict(l=250), showlegend=False
    )
    fig.show()

# --- PART B: Actual ordered→arrived durations + Fleet vs Infleeting split ---
fleet_data = df.copy()
fleet_data['actual_ordered_to_arrived'] = (
    fleet_data['med_ordered_to_produced'].fillna(0) +
    fleet_data['med_produced_to_arrived'].fillna(0)
)
fleet_data['actual_arrived_to_sub'] = (
    fleet_data['med_arrived_to_techprep'].fillna(0) +
    fleet_data['med_techprep_to_ready'].fillna(0) +
    fleet_data['med_ready_to_sub'].fillna(0)
)
fleet_data['actual_total'] = fleet_data['actual_ordered_to_arrived'] + fleet_data['actual_arrived_to_sub']
fleet_data['fleet_pct_of_total'] = (fleet_data['actual_ordered_to_arrived'] / fleet_data['actual_total'] * 100).round(1)
fleet_data['label'] = fleet_data['brand'] + ' @ ' + fleet_data['compound'].str.replace('_', ' ')
fleet_sorted = fleet_data.sort_values('actual_ordered_to_arrived', ascending=True)

print('
=== FLEET vs INFLEETING: Who Owns the Lifecycle Time? ===
')
print(f'{"Brand":<20} {"Compound":<30} {"Ord→Arr":>8} {"Arr→Sub":>8} {"Total":>8} {"Fleet %":>8}')
print('-' * 82)
for _, r in fleet_sorted.iterrows():
    fleet_flag = ' ⚠️' if r['fleet_pct_of_total'] > 70 else ''
    print(f'{r["brand"]:<20} {r["compound"]:<30} {r["actual_ordered_to_arrived"]:>7.0f}d {r["actual_arrived_to_sub"]:>7.0f}d {r["actual_total"]:>7.0f}d {r["fleet_pct_of_total"]:>7.1f}%{fleet_flag}')

print(f'
⚠️ = Fleet phase is >70% of total lifecycle (ETA accuracy dominates)')
print(f'Brands where Fleet >70%: {(fleet_data["fleet_pct_of_total"] > 70).sum()} / {len(fleet_data)}')

# Stacked bar: Fleet vs Infleeting
fig = go.Figure()
fig.add_trace(go.Bar(
    y=fleet_sorted['label'], x=fleet_sorted['actual_ordered_to_arrived'],
    name='Fleet Phase (ordered→arrived)', orientation='h',
    marker_color='#e45756', text=fleet_sorted['actual_ordered_to_arrived'].round(0).astype(int),
    textposition='inside'
))
fig.add_trace(go.Bar(
    y=fleet_sorted['label'], x=fleet_sorted['actual_arrived_to_sub'],
    name='Infleeting Phase (arrived→subscription)', orientation='h',
    marker_color='#4c78a8', text=fleet_sorted['actual_arrived_to_sub'].round(0).astype(int),
    textposition='inside'
))
fig.update_layout(
    barmode='stack', title='Fleet vs Infleeting: Who Owns the Lifecycle Time?',
    xaxis_title='Median Days', height=max(500, len(fleet_sorted) * 22),
    legend=dict(orientation='h', yanchor='bottom', y=1.02),
    margin=dict(l=250)
)
fig.show()


### 9.2 Infleeting Layer — Compound Lead Time Accuracy (Infleeting-owned)

The API's compound lead time (`t_arrived`) predicts how many days from compound arrival until subscription.  
This is **Infleeting's responsibility** and the only comparison that directly measures compound processing prediction quality.  
The ordered/produced predictions additionally depend on Fleet ETA accuracy (section 9.1).

In [ ]:
# === PART 1: Compound lead time accuracy (arrived → subscription) ===
# This is the meaningful comparison: API t_arrived vs actual total days from arrival to subscription

comparison = df.merge(expected_df, on=['brand', 'compound'], how='left')

# Compute actual arrived→subscription (sum of arrived→techprep + techprep→ready + ready→sub)
comparison['actual_arrived_to_sub'] = (
    comparison['med_arrived_to_techprep'].fillna(0) +
    comparison['med_techprep_to_ready'].fillna(0) +
    comparison['med_ready_to_sub'].fillna(0)
)

compound_accuracy = comparison[comparison['t_arrived'].notna()].copy()
compound_accuracy['delta_compound'] = compound_accuracy['actual_arrived_to_sub'] - compound_accuracy['t_arrived']
compound_accuracy['pct_error_compound'] = (compound_accuracy['delta_compound'] / compound_accuracy['t_arrived'] * 100).round(1)
compound_accuracy['label'] = compound_accuracy['brand'] + ' @ ' + compound_accuracy['compound']

print('=== COMPOUND LEAD TIME ACCURACY (t_arrived vs actual arrived→subscription) ===\n')
print(f'Comparisons: {len(compound_accuracy)}')
print(f'Mean absolute error: {compound_accuracy["delta_compound"].abs().mean():.1f} days')
print(f'Accurate (within ±20%): {(compound_accuracy["pct_error_compound"].abs() <= 20).sum()} / {len(compound_accuracy)} ({(compound_accuracy["pct_error_compound"].abs() <= 20).mean()*100:.0f}%)')
print(f'Faster than predicted: {(compound_accuracy["delta_compound"] < 0).sum()}')
print(f'Slower than predicted: {(compound_accuracy["delta_compound"] > 0).sum()}')

print(f'\nTop 15 compound lead time misses:')
compound_misses = compound_accuracy.sort_values('delta_compound', key=abs, ascending=False).head(15)
compound_misses[['brand', 'compound', 't_arrived', 'actual_arrived_to_sub', 'delta_compound', 'pct_error_compound', 'cars']]

### 9.3 Infleeting Layer — Compound Lead Time Heatmap (API t_arrived vs Actual)

In [ ]:
# Compound lead time heatmap: API prediction vs reality
ca = compound_accuracy.sort_values('delta_compound', ascending=True).copy()

fig = go.Figure()
fig.add_trace(go.Bar(
    y=ca['label'], x=ca['t_arrived'], name='API t_arrived (predicted)',
    orientation='h', marker_color='#4e79a7', opacity=0.7,
))
fig.add_trace(go.Bar(
    y=ca['label'], x=ca['actual_arrived_to_sub'], name='Actual arrived→sub (median)',
    orientation='h', marker_color='#e15759', opacity=0.7,
))
fig.update_layout(
    barmode='group',
    title='Compound Lead Time: API Prediction vs Reality (arrived→subscription)',
    xaxis_title='Days',
    height=max(600, len(ca) * 25),
    width=1000,
    legend=dict(orientation='h', yanchor='bottom', y=1.02),
)
fig.show()

# Also show the delta as a diverging bar
fig2 = go.Figure()
colors = ['#e15759' if d > 0 else '#4e79a7' for d in ca['delta_compound']]
fig2.add_trace(go.Bar(
    y=ca['label'], x=ca['delta_compound'],
    orientation='h', marker_color=colors,
    text=ca['delta_compound'].apply(lambda x: f'+{x:.0f}d' if x > 0 else f'{x:.0f}d'),
    textposition='outside',
))
fig2.update_layout(
    title='Compound Lead Time Error (actual − predicted)<br><sub>Red = slower than predicted, Blue = faster</sub>',
    xaxis_title='Delta (days)',
    height=max(600, len(ca) * 25),
    width=900,
)
fig2.show()

### 9.4 Per-Phase Accuracy (derived phase durations from t-minus differences)

**Ownership split**:
- `Ordered→Produced` and `Produced→Arrived`: **Fleet-owned** — derived expected = 0 for most brands because the API relies on Fleet ETA, not compound lead times
- `Arrived→Tech Prep`, `Tech Prep→Ready`, `Ready→Subscription`: **Infleeting-owned** — these are directly controlled by hardcoded compound lead times

Large deltas in Fleet phases are ETA accuracy problems. Large deltas in Infleeting phases are compound lead time errors we can fix.

In [ ]:
# Per-phase accuracy using derived phase durations (t-minus differences)
phase_mappings = [
    ('med_ordered_to_produced', 'exp_ordered_to_produced', 'Ordered→Produced'),
    ('med_produced_to_arrived', 'exp_produced_to_arrived', 'Produced→Arrived'),
    ('med_arrived_to_techprep', 'exp_arrived_to_techprep', 'Arrived→Tech Prep'),
    ('med_techprep_to_ready', 'exp_techprep_to_ready', 'Tech Prep→Ready'),
    ('med_ready_to_sub', 'exp_ready_to_sub', 'Ready→Subscription'),
]

accuracy_rows = []
for _, row in comparison.iterrows():
    if pd.isna(row.get('exp_ordered_to_produced')):
        continue
    for actual_col, expected_col, phase_name in phase_mappings:
        actual = row[actual_col]
        expected = row[expected_col]
        if pd.notna(actual) and pd.notna(expected) and expected > 0:
            delta = actual - expected
            pct_error = (delta / expected) * 100
            accuracy_rows.append({
                'brand': row['brand'], 'compound': row['compound'],
                'phase': phase_name, 'expected_days': expected,
                'actual_median': actual, 'delta_days': delta,
                'pct_error': round(pct_error, 1), 'cars': row['cars'],
            })

accuracy_df = pd.DataFrame(accuracy_rows)

# Phase accuracy summary
phase_order = ['Ordered→Produced', 'Produced→Arrived', 'Arrived→Tech Prep', 'Tech Prep→Ready', 'Ready→Subscription']
phase_acc = accuracy_df.groupby('phase').agg({
    'delta_days': ['mean', 'std', lambda x: x.abs().mean()],
    'pct_error': ['mean', lambda x: x.abs().mean()],
}).round(1)
phase_acc.columns = ['mean_delta', 'std_delta', 'mae_days', 'mean_pct_error', 'mape']
phase_acc = phase_acc.reindex([p for p in phase_order if p in phase_acc.index])

print('=== PER-PHASE ACCURACY (derived from t-minus differences) ===\n')
print('⚠️  Ordered→Produced: expected=0 for most brands (API has same ordered/produced values)')
print('    Non-zero only for Kia (14d) and Nissan (14d) where API values differ.\n')
phase_acc

### 9.5 Accuracy Heatmap — Per-Phase Delta (actual − expected)

In [ ]:
# Per-phase heatmap (only phases where expected > 0)
accuracy_pivot = accuracy_df.pivot_table(
    index=['brand', 'compound'], columns='phase', values='delta_days', aggfunc='first'
)
phase_order_available = [p for p in phase_order if p in accuracy_pivot.columns]
accuracy_pivot = accuracy_pivot[phase_order_available]
accuracy_pivot_labels = [f'{b} @ {c}' for b, c in accuracy_pivot.index]

fig = px.imshow(
    accuracy_pivot.values,
    labels=dict(x='Phase', y='Brand @ Compound', color='Delta (days)'),
    x=accuracy_pivot.columns.tolist(),
    y=accuracy_pivot_labels,
    color_continuous_scale='RdBu_r',
    color_continuous_midpoint=0,
    aspect='auto', text_auto=True,
)
fig.update_layout(
    title='Per-Phase Prediction Error: Actual − Expected (days)<br><sub>Red = slower than predicted, Blue = faster | Ordered→Produced only shows Kia/Nissan (others have expected=0)</sub>',
    height=max(600, len(accuracy_pivot) * 22),
    width=950, font=dict(size=11),
)
fig.show()

### 9.6 Recommendations — Compound Lead Times to Update (Infleeting-owned)

Ranked by impact: absolute error × volume. Only includes **Infleeting-owned** comparisons (arrived→subscription) where the API value is directly actionable by Infleeting.

In [ ]:
# Recommendations: compound lead time updates ranked by impact
recs = compound_accuracy[compound_accuracy['delta_compound'].abs() > 3].copy()
recs['abs_delta'] = recs['delta_compound'].abs()
recs['impact'] = recs['abs_delta'] * recs['cars']  # days-off × volume
recs = recs.sort_values('impact', ascending=False)

recs['direction'] = recs['delta_compound'].apply(lambda d: 'SLOWER than predicted' if d > 0 else 'FASTER than predicted')
recs['recommendation'] = recs.apply(
    lambda r: f"t_arrived: {int(r['t_arrived'])}d → {int(r['actual_arrived_to_sub'])}d", axis=1
)

print(f'Compound lead times needing updates (>3d off): {len(recs)}\n')
print('Top 20 by impact (error × volume):\n')
recs[['brand', 'compound', 't_arrived', 'actual_arrived_to_sub', 'delta_compound', 
      'direction', 'cars', 'recommendation']].head(20)

## 10. Lifecycle State Skip Analysis

Some brands routinely skip lifecycle states (e.g., `tech_prep_done` for MG, BYD, BMW/MINI). When a state is skipped, the hardcoded lead time for that phase is meaningless — cars jump directly to the next state, collapsing two phases into one.

This section quantifies **skip rates per brand/compound** to identify where predictions should be adjusted or states removed from the model.

In [ ]:
skip_query = f"""
WITH car_states AS (
  SELECT
    car_id,
    MAX(IF(name='car_ordered', JSON_VALUE(snapshot, '$.oem'), NULL)) AS oem,
    MAX(IF(name='car_state_changed_arrived_from_supplier',
           JSON_VALUE(snapshot, '$.infleeting_compound_id'), NULL)) AS compound_id,
    MAX(IF(name='car_ordered', 1, 0)) AS has_ordered,
    MAX(IF(name='car_state_changed_produced', 1, 0)) AS has_produced,
    MAX(IF(name='car_state_changed_arrived_from_supplier', 1, 0)) AS has_arrived,
    MAX(IF(name='car_state_changed_tech_prep_done', 1, 0)) AS has_techprep,
    MAX(IF(name='car_state_changed_ready_to_deliver', 1, 0)) AS has_ready,
    MAX(IF(name='car_state_changed_in_subscription', 1, 0)) AS has_sub
  FROM `datawarehouse-304513.cars_ops_postgres_public.car_events`
  WHERE name IN (
    'car_ordered','car_state_changed_produced','car_state_changed_arrived_from_supplier',
    'car_state_changed_tech_prep_done','car_state_changed_ready_to_deliver',
    'car_state_changed_in_subscription'
  )
  AND time >= '2025-01-01' AND time < '2026-01-01'
  GROUP BY car_id
)
SELECT
  oem AS brand,
  compound_id AS compound,
  COUNT(*) AS total_cars,
  SUM(has_ordered) AS with_ordered,
  SUM(has_produced) AS with_produced,
  SUM(has_arrived) AS with_arrived,
  SUM(has_techprep) AS with_techprep,
  SUM(has_ready) AS with_ready,
  SUM(has_sub) AS with_sub,
  -- Skip rates for key transitions
  ROUND(SAFE_DIVIDE(SUM(IF(has_arrived = 1 AND has_techprep = 0 AND has_ready = 1, 1, 0)),
        SUM(IF(has_arrived = 1 AND has_ready = 1, 1, 0))) * 100, 1) AS pct_skip_techprep,
  ROUND(SAFE_DIVIDE(SUM(IF(has_produced = 0 AND has_arrived = 1, 1, 0)),
        SUM(IF(has_arrived = 1, 1, 0))) * 100, 1) AS pct_skip_produced,
  ROUND(SAFE_DIVIDE(SUM(IF(has_ordered = 1 AND has_produced = 0 AND has_arrived = 1, 1, 0)),
        SUM(IF(has_ordered = 1 AND has_arrived = 1, 1, 0))) * 100, 1) AS pct_ordered_skip_produced
FROM car_states
WHERE (oem, compound_id) IN ({filter_tuples})
GROUP BY 1, 2
HAVING total_cars >= 5
ORDER BY pct_skip_techprep DESC
"""

skip_df = run_query(skip_query)

# Highlight problematic skip rates
high_skip_tp = skip_df[skip_df['pct_skip_techprep'] > 20].copy()
print(f'Brand/compound combos with >20% tech_prep skip rate: {len(high_skip_tp)}')
print()

fig = px.bar(
    skip_df.sort_values('pct_skip_techprep', ascending=True),
    x='pct_skip_techprep',
    y=skip_df.sort_values('pct_skip_techprep', ascending=True).apply(
        lambda r: f"{r['brand']} @ {r['compound']}", axis=1),
    orientation='h',
    title='Tech Prep Done Skip Rate by Brand/Compound (2025)',
    labels={'pct_skip_techprep': '% Cars Skipping Tech Prep', 'y': ''},
    text='pct_skip_techprep',
)
fig.update_layout(
    height=max(500, len(skip_df) * 20), width=900,
    xaxis=dict(range=[0, 100]),
)
fig.show()

print('\nSkip rates summary:')
skip_df[['brand', 'compound', 'total_cars', 'pct_skip_techprep', 'pct_skip_produced', 'pct_ordered_skip_produced']]

## 11. Distribution Analysis — Percentile Windows per Phase

Median alone hides the spread. To build calibrated windows that hit 95-99% accuracy, we need the full distribution: P5, P25, P50, P75, P95, P99.

This reveals:
- **Tight distributions** (P5 ≈ P95): predictable, safe to use median
- **Wide distributions** (P95 >> P50): high variance, need wider windows or noise filtering
- **Heavy right tails** (P99 >> P95): outliers from financing blockers, call-off delays, etc.

In [ ]:
pctl_query = f"""
WITH events_all AS (
  SELECT
    e.car_id, e.name, e.time,
    JSON_VALUE(e.snapshot, '$.oem') AS oem,
    JSON_VALUE(e.snapshot, '$.infleeting_compound_id') AS compound_id
  FROM `datawarehouse-304513.cars_ops_postgres_public.car_events` e
  WHERE e.name IN (
    'car_ordered','car_state_changed_produced','car_state_changed_arrived_from_supplier',
    'car_state_changed_tech_prep_done','car_state_changed_ready_to_deliver','car_state_changed_in_subscription'
  )
),
cars_active_2025 AS (
  SELECT DISTINCT car_id FROM events_all
  WHERE time >= '2025-01-01' AND time < '2026-01-01'
),
filtered AS (
  SELECT e.* FROM events_all e
  INNER JOIN cars_active_2025 c ON e.car_id = c.car_id
),
state_changes AS (
  SELECT car_id, name, time, oem, compound_id,
    LAG(time) OVER (PARTITION BY car_id ORDER BY time) AS prev_time
  FROM filtered
),
with_days AS (
  SELECT *, DATE_DIFF(DATE(time), DATE(prev_time), DAY) AS days_in_status
  FROM state_changes WHERE prev_time IS NOT NULL
),
compound_lookup AS (
  SELECT DISTINCT car_id, compound_id FROM filtered
  WHERE name = 'car_state_changed_arrived_from_supplier' AND compound_id IS NOT NULL
),
enriched AS (
  SELECT
    d.car_id, d.name, d.days_in_status, d.oem AS brand,
    COALESCE(cl.compound_id, d.compound_id) AS compound
  FROM with_days d
  LEFT JOIN compound_lookup cl ON d.car_id = cl.car_id
  WHERE (d.oem, COALESCE(cl.compound_id, d.compound_id)) IN ({filter_tuples})
)
SELECT
  brand, compound,
  CASE name
    WHEN 'car_state_changed_produced' THEN 'Ordered→Produced'
    WHEN 'car_state_changed_arrived_from_supplier' THEN 'Produced→Arrived'
    WHEN 'car_state_changed_tech_prep_done' THEN 'Arrived→Tech Prep'
    WHEN 'car_state_changed_ready_to_deliver' THEN 'Tech Prep→Ready'
    WHEN 'car_state_changed_in_subscription' THEN 'Ready→Subscription'
  END AS phase,
  COUNT(*) AS n,
  APPROX_QUANTILES(days_in_status, 100 IGNORE NULLS)[OFFSET(5)] AS p5,
  APPROX_QUANTILES(days_in_status, 100 IGNORE NULLS)[OFFSET(25)] AS p25,
  APPROX_QUANTILES(days_in_status, 100 IGNORE NULLS)[OFFSET(50)] AS p50,
  APPROX_QUANTILES(days_in_status, 100 IGNORE NULLS)[OFFSET(75)] AS p75,
  APPROX_QUANTILES(days_in_status, 100 IGNORE NULLS)[OFFSET(95)] AS p95,
  APPROX_QUANTILES(days_in_status, 100 IGNORE NULLS)[OFFSET(99)] AS p99
FROM enriched
WHERE days_in_status >= 0
GROUP BY 1, 2, 3
HAVING n >= 5
ORDER BY brand, compound, phase
"""

pctl_df = run_query(pctl_query)
print(f'{len(pctl_df)} brand/compound/phase rows with percentile data')

# Flag wide distributions (P95/P50 > 3x = heavy tail)
pctl_df['spread_ratio'] = (pctl_df['p95'] / pctl_df['p50'].replace(0, 1)).round(1)
pctl_df['tail_ratio'] = (pctl_df['p99'] / pctl_df['p95'].replace(0, 1)).round(1)
pctl_df['window_95'] = pctl_df['p95'] - pctl_df['p5']  # 90% confidence window width

wide_dist = pctl_df[pctl_df['spread_ratio'] > 3].sort_values('spread_ratio', ascending=False)
print(f'\nPhases with P95/P50 > 3x (unpredictable): {len(wide_dist)}')
if len(wide_dist) > 0:
    print(wide_dist[['brand', 'compound', 'phase', 'n', 'p5', 'p50', 'p95', 'p99', 'spread_ratio']].head(20).to_string(index=False))

# Visualize distribution widths
pctl_df['label'] = pctl_df['brand'] + ' @ ' + pctl_df['compound']
pctl_df

In [ ]:
# Box-plot style visualization: P5-P95 range with P50 marker per phase
# Focus on compound phases (arrived onwards) where predictions matter most

compound_phases = ['Arrived→Tech Prep', 'Tech Prep→Ready', 'Ready→Subscription']
compound_pctl = pctl_df[pctl_df['phase'].isin(compound_phases)].copy()

fig = go.Figure()
for _, row in compound_pctl.sort_values(['phase', 'label']).iterrows():
    fig.add_trace(go.Box(
        name=f"{row['label']} | {row['phase']}",
        lowerfence=[row['p5']],
        q1=[row['p25']],
        median=[row['p50']],
        q3=[row['p75']],
        upperfence=[row['p95']],
        orientation='h',
    ))

fig.update_layout(
    title='Compound Phase Distributions — P5 to P95 Range (2025)',
    xaxis_title='Days',
    height=max(600, len(compound_pctl) * 25),
    width=1000,
    showlegend=False,
)
fig.show()

# Summary: which brand/compound combos have the tightest vs widest windows?
print('=== Tightest 90% windows (most predictable) ===')
tight = pctl_df[pctl_df['phase'].isin(compound_phases)].nsmallest(10, 'window_95')
print(tight[['brand', 'compound', 'phase', 'n', 'p5', 'p50', 'p95', 'window_95']].to_string(index=False))

print('\n=== Widest 90% windows (least predictable) ===')
wide = pctl_df[pctl_df['phase'].isin(compound_phases)].nlargest(10, 'window_95')
print(wide[['brand', 'compound', 'phase', 'n', 'p5', 'p50', 'p95', 'window_95']].to_string(index=False))

## 12. Granularity Comparison — Brand-Level vs. Brand+Compound-Level

Current predictions use Brand+Compound+State granularity. But is compound-level granularity actually needed, or would brand-level be sufficient?

This compares prediction accuracy at two levels:
1. **Brand-level**: same expected lead time for all compounds of a brand
2. **Brand+Compound-level**: different expected lead times per compound (current approach)

If brand+compound produces significantly better accuracy, the granularity is justified.

In [ ]:
# Compare brand-level vs brand+compound-level prediction accuracy
# Brand-level: use the brand's average median across all compounds as the "expected"

phase_cols_med = [
    'med_ordered_to_produced', 'med_produced_to_arrived',
    'med_arrived_to_techprep', 'med_techprep_to_ready', 'med_ready_to_sub'
]

# Compute brand-level medians (average of compound medians, weighted by cars)
brand_medians = df.groupby('brand').apply(
    lambda g: pd.Series({
        col: (g[col] * g['cars']).sum() / g['cars'].sum()
        for col in phase_cols_med
    })
).reset_index()

# For each brand/compound, compute error vs brand-level prediction
brand_level_errors = []
compound_level_errors = []

for _, row in comparison.iterrows():
    if pd.isna(row.get('exp_ordered_to_produced')):
        continue
    brand = row['brand']
    brand_med = brand_medians[brand_medians['brand'] == brand]
    if len(brand_med) == 0:
        continue
    brand_med = brand_med.iloc[0]
    
    for actual_col, expected_col, phase_name in phase_mappings:
        actual = row[actual_col]
        expected_compound = row[expected_col]
        expected_brand = brand_med[actual_col]
        
        if pd.notna(actual) and pd.notna(expected_compound) and expected_compound > 0:
            compound_level_errors.append({
                'brand': brand,
                'compound': row['compound'],
                'phase': phase_name,
                'actual': actual,
                'expected': expected_compound,
                'error': abs(actual - expected_compound),
                'level': 'Brand+Compound',
            })
        if pd.notna(actual) and pd.notna(expected_brand) and expected_brand > 0:
            brand_level_errors.append({
                'brand': brand,
                'compound': row['compound'],
                'phase': phase_name,
                'actual': actual,
                'expected': expected_brand,
                'error': abs(actual - expected_brand),
                'level': 'Brand Only',
            })

compound_err_df = pd.DataFrame(compound_level_errors)
brand_err_df = pd.DataFrame(brand_level_errors)

# Compare MAE
print('=== Granularity Comparison: Mean Absolute Error ===\n')
print(f'Brand+Compound level MAE: {compound_err_df["error"].mean():.1f} days')
print(f'Brand-only level MAE:     {brand_err_df["error"].mean():.1f} days')
print(f'Improvement from compound granularity: {brand_err_df["error"].mean() - compound_err_df["error"].mean():.1f} days')

# Compare by phase
print('\n--- MAE by Phase ---')
for phase in ['Ordered→Produced', 'Produced→Arrived', 'Arrived→Tech Prep', 'Tech Prep→Ready', 'Ready→Subscription']:
    c_mae = compound_err_df[compound_err_df['phase'] == phase]['error'].mean()
    b_mae = brand_err_df[brand_err_df['phase'] == phase]['error'].mean()
    better = 'Compound' if c_mae < b_mae else 'Brand'
    print(f'  {phase:25s}  Compound: {c_mae:5.1f}d  Brand: {b_mae:5.1f}d  → {better} wins by {abs(c_mae-b_mae):.1f}d')

# Which brands benefit most from compound granularity?
print('\n--- Brands where compound granularity matters most ---')
both = pd.merge(
    compound_err_df.groupby('brand')['error'].mean().rename('compound_mae'),
    brand_err_df.groupby('brand')['error'].mean().rename('brand_mae'),
    left_index=True, right_index=True
)
both['improvement'] = both['brand_mae'] - both['compound_mae']
both = both.sort_values('improvement', ascending=False)
print(both.round(1).to_string())

## 13. Data-Driven Lead Times & Calibrated Windows

Replace static hardcoded values with **data-driven lead times** and **calibrated deviation windows** that target 95% accuracy.

For each brand/compound/phase:
- **Point estimate**: P50 (median) — best single guess
- **Optimistic bound**: P25 — "if things go well"
- **Conservative bound**: P75 — "realistic buffer"
- **95% window**: [P5, P95] — should capture 90% of cars
- **99% window**: [P1, P99] — should capture 98% of cars

This produces a ready-to-use replacement table for the current static lead times.

In [ ]:
# Build the replacement table: data-driven lead times with calibrated windows
# Merge percentile data with current expected values for comparison

phase_order = ['Ordered→Produced', 'Produced→Arrived', 'Arrived→Tech Prep', 'Tech Prep→Ready', 'Ready→Subscription']
exp_col_map = {
    'Ordered→Produced': 'exp_ordered_to_produced',
    'Produced→Arrived': 'exp_produced_to_arrived',
    'Arrived→Tech Prep': 'exp_arrived_to_techprep',
    'Tech Prep→Ready': 'exp_techprep_to_ready',
    'Ready→Subscription': 'exp_ready_to_sub',
}

# Merge expected values into percentile df
calibrated_rows = []
for _, prow in pctl_df.iterrows():
    brand, compound, phase = prow['brand'], prow['compound'], prow['phase']
    exp_row = expected_df[(expected_df['brand'] == brand) & (expected_df['compound'] == compound)]
    
    current_expected = None
    if len(exp_row) > 0 and phase in exp_col_map:
        current_expected = exp_row.iloc[0].get(exp_col_map[phase])
    
    # Check skip rate for this brand/compound
    skip_row = skip_df[(skip_df['brand'] == brand) & (skip_df['compound'] == compound)] if 'skip_df' in dir() else pd.DataFrame()
    skip_pct = skip_row.iloc[0]['pct_skip_techprep'] if len(skip_row) > 0 else None
    
    calibrated_rows.append({
        'brand': brand,
        'compound': compound,
        'phase': phase,
        'n': prow['n'],
        'current_expected': current_expected,
        'p5': prow['p5'],
        'p25': prow['p25'],
        'p50_recommended': prow['p50'],
        'p75': prow['p75'],
        'p95': prow['p95'],
        'p99': prow['p99'],
        'window_90pct': f"[{prow['p5']}–{prow['p95']}]",
        'delta_vs_current': prow['p50'] - current_expected if pd.notna(current_expected) else None,
        'skip_techprep_pct': skip_pct if phase in ['Arrived→Tech Prep', 'Tech Prep→Ready'] else None,
    })

calibrated_df = pd.DataFrame(calibrated_rows)

# Show the replacement table
print('=== Data-Driven Lead Times — Replacement Table ===\n')
print('current_expected = t-minus derived phase duration')
print('p50_recommended = historical median (recommended replacement)')
print('window_90pct = [P5, P95] range capturing 90% of cars\n')

# Focus on compound phases where predictions matter
compound_cal = calibrated_df[calibrated_df['phase'].isin(compound_phases)].copy()
compound_cal = compound_cal.sort_values(['brand', 'compound', 'phase'])
compound_cal[['brand', 'compound', 'phase', 'n', 'current_expected', 'p50_recommended', 
              'delta_vs_current', 'window_90pct', 'skip_techprep_pct']]

In [ ]:
# Visualize: current expected vs data-driven P50 with P5-P95 error bars
# For compound phases only

for phase in compound_phases:
    phase_data = calibrated_df[
        (calibrated_df['phase'] == phase) & 
        calibrated_df['current_expected'].notna()
    ].sort_values('p50_recommended')
    
    if len(phase_data) == 0:
        continue
    
    labels = phase_data.apply(lambda r: f"{r['brand']} @ {r['compound']}", axis=1)
    
    fig = go.Figure()
    
    # P5-P95 range (error bars)
    fig.add_trace(go.Scatter(
        x=phase_data['p50_recommended'],
        y=labels,
        error_x=dict(
            type='data',
            symmetric=False,
            array=phase_data['p95'] - phase_data['p50_recommended'],
            arrayminus=phase_data['p50_recommended'] - phase_data['p5'],
        ),
        mode='markers',
        marker=dict(size=10, color='#4e79a7'),
        name='Data-Driven P50 [P5–P95]',
    ))
    
    # Current expected (red markers)
    fig.add_trace(go.Scatter(
        x=phase_data['current_expected'],
        y=labels,
        mode='markers',
        marker=dict(size=10, symbol='x', color='red'),
        name='Current Hardcoded',
    ))
    
    fig.update_layout(
        title=f'{phase}: Current vs Data-Driven Lead Times',
        xaxis_title='Days',
        height=max(400, len(phase_data) * 25),
        width=900,
        legend=dict(orientation='h', yanchor='bottom', y=1.02),
    )
    fig.show()

### 13.1 Summary: Key Findings & Recommendations

**Interpretation guide for the replacement table above:**
- `delta_vs_current > 0`: current prediction is too optimistic (cars take longer)
- `delta_vs_current < 0`: current prediction is too pessimistic (cars arrive faster)
- `skip_techprep_pct > 50%`: this brand regularly skips tech_prep — consider removing it from the model
- `window_90pct` width > 20 days: high variance, single point estimate is unreliable

In [ ]:
# Final summary: actionable recommendations
print('=' * 80)
print('ACTIONABLE RECOMMENDATIONS')
print('=' * 80)

# 1. Phases to update
print('\n1. LEAD TIMES TO UPDATE (current off by >3 days, compound phases only):')
updates = calibrated_df[
    (calibrated_df['phase'].isin(compound_phases)) &
    (calibrated_df['delta_vs_current'].notna()) &
    (calibrated_df['delta_vs_current'].abs() > 3)
].sort_values('delta_vs_current', key=abs, ascending=False)

if len(updates) > 0:
    for _, r in updates.iterrows():
        direction = 'TOO LOW' if r['delta_vs_current'] > 0 else 'TOO HIGH'
        print(f"  {r['brand']:15s} @ {r['compound']:30s} | {r['phase']:20s} | "
              f"current: {r['current_expected']:4.0f}d → recommended: {r['p50_recommended']:4.0f}d "
              f"({direction}, off by {abs(r['delta_vs_current']):.0f}d) "
              f"[P5-P95: {r['p5']}-{r['p95']}d, n={r['n']}]")

# 2. States to skip
print('\n2. LIFECYCLE STATES TO CONSIDER REMOVING (>50% skip rate):')
if 'skip_df' in dir():
    high_skip = skip_df[skip_df['pct_skip_techprep'] > 50]
    for _, r in high_skip.iterrows():
        print(f"  {r['brand']:15s} @ {r['compound']:30s} | tech_prep skip rate: {r['pct_skip_techprep']:.0f}% ({r['total_cars']} cars)")

# 3. High-variance phases (unreliable single-point predictions)
print('\n3. HIGH-VARIANCE PHASES (90% window > 30 days — need dynamic windows):')
high_var = calibrated_df[
    (calibrated_df['phase'].isin(compound_phases)) &
    ((calibrated_df['p95'] - calibrated_df['p5']) > 30)
].sort_values('p95', ascending=False)
for _, r in high_var.iterrows():
    print(f"  {r['brand']:15s} @ {r['compound']:30s} | {r['phase']:20s} | "
          f"P50: {r['p50_recommended']:.0f}d, but 90% window: [{r['p5']:.0f}–{r['p95']:.0f}]d (spread: {r['p95']-r['p5']:.0f}d, n={r['n']})")

# 4. Fleet vs Infleeting ownership summary
print('\n4. FLEET vs INFLEETING OWNERSHIP SUMMARY:')
print('   Fleet-owned (ordered→arrived): ETA accuracy — no prediction data in car_events to compare')
print('   Infleeting-owned (arrived→subscription): compound lead time accuracy — directly testable\n')

# Show which brands have Fleet-dominated vs Infleeting-dominated error
for _, exp_r in expected_df[expected_df['t_arrived'].notna()].iterrows():
    brand, compound = exp_r['brand'], exp_r['compound']
    actual_row = df[(df['brand'] == brand) & (df['compound'] == compound)]
    if len(actual_row) == 0:
        continue
    ar = actual_row.iloc[0]
    fleet_actual = (ar.get('med_ordered_to_produced', 0) or 0) + (ar.get('med_produced_to_arrived', 0) or 0)
    infleeting_actual = (ar.get('med_arrived_to_techprep', 0) or 0) + (ar.get('med_techprep_to_ready', 0) or 0) + (ar.get('med_ready_to_sub', 0) or 0)
    infleeting_predicted = exp_r['t_arrived']
    infleeting_error = infleeting_actual - infleeting_predicted
    total = fleet_actual + infleeting_actual
    fleet_pct = (fleet_actual / total * 100) if total > 0 else 0
    if fleet_pct > 70 or abs(infleeting_error) > 10:
        owner = 'FLEET' if fleet_pct > 70 else 'INFLEETING'
        print(f"  {brand:15s} @ {compound:30s} | Fleet: {fleet_actual:.0f}d ({fleet_pct:.0f}%), Infleeting: {infleeting_actual:.0f}d (predicted {infleeting_predicted:.0f}d, Δ{infleeting_error:+.0f}d) → {owner} dominates"
        )